# GPT Use Case 2: Analyze the product sales

Import libraries and access API

In [ ]:
import openai
import os
import pandas as pd
import json

GPT_MODEL = "gpt-3.5-turbo-0613" # select model based on application
openai.api_key = API_key = 'sk-...'

Import sales from csv file

In [ ]:
# Import csv file
data_path = './GPT_sales_analysis_data'
sales_file = os.path.join(data_path, 'Sales_April_2019.csv')

# Use pandas to create DataFrame from csv file
df = pd.read_csv(sales_file)
df.dropna(inplace=True)
df = df.head(50) # batch the data to prevent rate limit errors
dict_sale = df.to_dict(orient='dict')

Create prompt template

In [ ]:
instruction = (\
    """You have no previous memories\n"""
    """You are a professional business analyst.\n"""
    """You task is to summarize each product sales.\n"""
    """You should also provide statistical information of each product such as 'total_sales', 'total_quantity', 'average_price', 'most_common_address'\n"""
    f"""Sales: {dict_sale}\n"""
    """Return the result into JSON schema\n"""
    )

Input prepared prompt to LLM and run GPT

In [ ]:
response = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=[
        {'role': 'user', "content": instruction}
    ],
    temperature=0
)

result = response['choices'][0]['message']['content']
print(result)

Load and post-process GPT response

In [ ]:
json_result = json.loads(result)
print(json_result)

Export csv file

In [ ]:
export_path = os.path.join('./GPT_sales_analysis_report', 'sales_summary.csv')
# Clear existed csv file
if os.path.exists(export_path):
    os.remove(export_path)

final_df = pd.DataFrame.from_dict(json_result['products']) # Convert dict to DataFrame
final_df.to_csv(export_path, encoding='utf-8-sig', index=False)
print(final_df)